# Team 17

# 109020039, 109020015, X1110071, X1110072

In [51]:
#Load libraries
import pandas as pd
import nltk
import matplotlib
import numpy
import warnings
warnings.filterwarnings("ignore")

In [30]:
df = pd.read_csv("small_df.csv",delimiter=',')
df.drop(index=[0], inplace=True)

In [31]:
all_doc = df.to_dict(orient='list')

# Take Subway for example

In [41]:
data = list(df['Subway'])

In [42]:
for i in range(1,5):
    data[i] = data[i][4:]
data

['I would like to mention menu for Vegetarians.\r\n\r\nLike in Subway, they can go for Falafel Sandwich which is very good option for them even they can try veg sandwich.\r\n\r\nAnd also you can try cookies which cost around 50 for 3 pieces.',
 "'Great food, but I think service can be better, the speed of serve and order aren’t quick enough.']",
 "'This is the nearest Subway to Section 2 Guangfu Road. This has all the latest Sub way items available in Taiwan. You can dine in/ take out.']",
 '"Pretty solid for Subway. You will have some issues finding parking for a car but on the other hand there\'s almost never any wait time. The place is also next to a711 which makes it convenient to get the same drink for 10 ntd less."]',
 '"What\'s with the attitude?\\nI asked for some extra veggies and it\'s totally fine if you couldn\'t offer me, but you shouldn\'t have just pretended that you didn\'t hear me.\\nI said THRICE clearly enough for you to hear."]']

# Apply first skill

In [45]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

C:\Users\Master\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
model_name = "textattack/roberta-base-SST-2" # This code snippet loads the textattack/roberta-base-SST-2 model, which is a RoBERTa model fine-tuned for the SST-2 (Stanford Sentiment Treebank) dataset.
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
def Roberta(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    sentiment_class = torch.argmax(logits).item()
    if sentiment_class == 1:
        print(f"The sentiment of the input text is positive.")
    else:
        print(f"The sentiment of the input text is negative.")
    return sentiment_class

In [60]:
pos = 0
neg = 0
for item in data:
    if(Roberta(item)==1):
        pos = pos+1
    else:
        neg = neg+1

print("The number of positive comments is",pos)
print("The number of negative comments is",neg)


The sentiment of the input text is positive.
The sentiment of the input text is negative.
The sentiment of the input text is positive.
The sentiment of the input text is positive.
The sentiment of the input text is negative.
The number of positive comments is 3
The number of negative comments is 2


# Concusion of first skill

The rank of second and fifith comments are 3 and 2, respectively, which is insistant with the above result (both of them are negative)

While all the others have higher rank, they are valued as positive.

Hence we can see that the model predicted correcticely in this case.

# Apply second skill

In [74]:
data2 = " ".join(data)
stopword = ["\\","[","]","\r","\n","'",'"',"/"]

data2 = [word for word  in data2 if word not in stopword]

data2 = "".join(data2)
print(data2)

I would like to mention menu for Vegetarians.Like in Subway, they can go for Falafel Sandwich which is very good option for them even they can try veg sandwich.And also you can try cookies which cost around 50 for 3 pieces. Great food, but I think service can be better, the speed of serve and order aren’t quick enough. This is the nearest Subway to Section 2 Guangfu Road. This has all the latest Sub way items available in Taiwan. You can dine in take out. Pretty solid for Subway. You will have some issues finding parking for a car but on the other hand theres almost never any wait time. The place is also next to a711 which makes it convenient to get the same drink for 10 ntd less. Whats with the attitude?nI asked for some extra veggies and its totally fine if you couldnt offer me, but you shouldnt have just pretended that you didnt hear me.nI said THRICE clearly enough for you to hear.


In [75]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [77]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [80]:
# Tokenize the input text and convert it to a PyTorch tensor
inputs = tokenizer(data2, return_tensors="pt", max_length=1024, truncation=True)

# Perform text summarization using the BART model
with torch.no_grad():
    outputs = model.generate(**inputs)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the result
print(f"Summary: {summary}")

Summary: Great food, but I think service can be better, the speed of serve and order aren’t quick enough. This is the nearest Subway to Section 2 Guangfu Road. You can dine in take out. Pretty solid for Subway. You will have some issues finding parking for a car but on the other hand theres almost never any wait time.


# Concusion for second skill

We conbine all the comments of this restaurant into a single string and remove some stopwords. 

And the summary provided overall thought of this restaurant.